In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time

In [4]:
import feather

In [3]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [5]:
PATH = "data/bulldozer/"

df_raw = feather.read_dataframe('tmp/bulldozers-raw')
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice')

In [6]:
def split_vals(a,n): return a[:n], a[n:]
n_valid = 12000
n_trn = len(df_trn)-n_valid
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)
raw_train, raw_valid = split_vals(df_raw, n_trn)

In [7]:
x_sub = X_train[['YearMade', 'MachineHoursCurrentMeter']]

In [8]:
class TreeEnsemble:
    def __init__(self, x, y, n_trees, sample_sz, min_leaf=5):
        np.random.seed(42)
        self.x, self.y, self.sample_sz, self.min_leaf = x, y, sample_sz, min_leaf
        self.trees = [self.create_tree() for i in range(n_trees)]
        
    def create_tree(self):
        rnd_idxs = np.random.permutation(len(self.y))[:self.sample_sz] # Sampling without replacement
        return DecisionTree(self.x.iloc[rnd_idxs], self.y[rnd_idxs], min_leaf=self.min_leaf)
    
    def predict(self, x):
        return np.mean([t.predict(x) for t in self.trees], axis=0)

In [9]:
class DecisionTree:
    def __init__(self, x, y, idxs=None, min_leaf=5):
        self.x, self.y, self.idxs, self.min_leaf = x, y, idxs, min_leaf

In [10]:
m = TreeEnsemble(X_train, y_train, n_trees=10, sample_sz=1000, min_leaf=3)

In [11]:
m.trees[0]

In [28]:
class DecisionTree:
    def __init__(self, x, y, idxs=None, min_leaf=5):
        if idxs is None: idxs=np.arange(len(y))
        self.x, self.y, self.idxs, self.min_leaf = x, y, idxs, min_leaf
        self.n, self.c = len(idxs), x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()
        
    def find_varsplit(self):
        for i in range(self.c):
            self.find_better_split(i)
            
    def find_better_split(self, var_idx):
        x,y = self.x.values[self.idxs,var_idx], self.y[self.idxs]

        for i in range(self.n):
            lhs = x<=x[i]
            rhs = x>x[i]
            if rhs.sum()<self.min_leaf or lhs.sum()<self.min_leaf: continue
            lhs_std = y[lhs].std()
            rhs_std = y[rhs].std()
            curr_score = lhs_std*lhs.sum() + rhs_std*rhs.sum()
            if curr_score<self.score: 
                self.var_idx,self.score,self.split = var_idx,curr_score,x[i]
    
    @property
    def split_name(self):
        return self.x.columns[self.var_idx]
    
    @property
    def split_col(self):
        return self.x.values[self.idxs, self.var_idxs]

    @property
    def is_leaf(self):
        return self.score == float('inf')
    
    def predict(self, y):
        return np.array([self.predict_row(yi) for yi in y])
    
    def predict_row(self, yi):
        if self.is_leaf: return self.val
        t = self.lhs if yi[self.var_idx] <= self.split else self.rhs
        return t.predict_row(yi)
    
    def __repr__(self):
        s = f'n: {self.n}; val: {self.val}'
        if not self.is_leaf:
            s += f'; score:{self.score}; split:{self.split}; var:{self.split_name}'
        return s
        

In [29]:
def find_better_split(self, var_idx):
    x,y = self.x.values[self.idxs,var_idx], self.y[self.idxs]

    for i in range(self.n):
        lhs = x<=x[i]
        rhs = x>x[i]
        if rhs.sum()<self.min_leaf or lhs.sum()<self.min_leaf: continue
        lhs_std = y[lhs].std()
        rhs_std = y[rhs].std()
        curr_score = lhs_std*lhs.sum() + rhs_std*rhs.sum()
        if curr_score<self.score: 
            self.var_idx,self.score,self.split = var_idx,curr_score,x[i]

In [30]:
m = TreeEnsemble(X_train, y_train, n_trees=10, sample_sz=1000, min_leaf=3)

In [31]:
m.trees[0]

n: 1000; val: 10.079014121552744; score:608.1609924487253; split:0; var:Coupler_System

In [32]:
%load_ext Cython

In [33]:
def fib1(n):
    a, b = 0, 1
    while b < n:
        a, b = b, a + b

In [34]:
%%cython
def fib2(n):
    a, b = 0, 1
    while b < n:
        a, b = b, a + b

In [35]:
%%cython
def fib3(int n):
    cdef int b = 1
    cdef int a = 0
    cdef int t = 0
    while b < n:
        t = a
        a = b
        b = a + b

In [37]:
%timeit fib1(50)
%timeit fib2(50)
%timeit fib3(50)

780 ns ± 11.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
301 ns ± 8.19 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
50.5 ns ± 0.701 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [42]:
%%cython
def fib4(int n):
    cdef int b = 1
    cdef int a = 0
    cdef int t = 0
    while b < n:
        t = a
        a = b
        b = a + b

In [43]:
%timeit fib4(50)

57.3 ns ± 0.374 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
